In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from pythonosc.osc_server import BlockingOSCUDPServer
from pythonosc.dispatcher import Dispatcher
from pythonosc.udp_client import SimpleUDPClient
import struct

In [2]:
ip = '127.0.0.1'
receive_port = 2323
send_port = 7531

In [3]:
class DataBlock:
    
    def __init__(self, n_rows, n_cols):
        self.data = np.zeros((n_rows, n_cols))
        
    def data_handler(self, address, byte_array):
        
        float_list = struct.unpack(self.data.size*'f', byte_array)
        self.data = np.array(float_list).reshape(*self.data.shape)
        
def default_handler(address, *args):
    print(f'DEFAULT {address}: {args}')

In [4]:
# Instantiate dispatcher
dispatcher = Dispatcher()

# Setup data class. Make sure the size matches the
data_block = DataBlock(3, 256)

# Map handler functions to message types
dispatcher.map('/eeg', data_block.data_handler)
dispatcher.set_default_handler(default_handler)

# Setup server and client
server = BlockingOSCUDPServer((ip, receive_port), dispatcher)
client = SimpleUDPClient(ip, send_port)

# Read 20 blocks
for i in range(20):
    server.handle_request()
    # Do something with data_block.data here

# Example message send
client.send_message("/number", 1)